In [4]:
import os
import shutil
import random

# Define paths
dataset_root = "../data/dataset_split/train"  # Original dataset
subset_root = "../data/subset/train"  # New small subset

# Define how many images to copy per category
num_images_per_class = 10  # Adjust as needed

# Ensure the subset directory exists
os.makedirs(subset_root, exist_ok=True)

# Loop through each category and copy images
for category in os.listdir(dataset_root):
    class_path = os.path.join(dataset_root, category)
    subset_class_path = os.path.join(subset_root, category)

    os.makedirs(subset_class_path, exist_ok=True)

    # Get a list of images in this category
    images = os.listdir(class_path)
    
    # Select a random subset
    selected_images = random.sample(images, min(num_images_per_class, len(images)))

    # Copy selected images to new folder
    for img_name in selected_images:
        src_path = os.path.join(class_path, img_name)
        dst_path = os.path.join(subset_class_path, img_name)
        shutil.copy(src_path, dst_path)

print(f"✅ Small dataset subset created at {subset_root}!")


✅ Small dataset subset created at ../data/subset/train!


In [14]:
! nvidia-smi


Wed Feb 19 19:16:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.03                 Driver Version: 561.03         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   61C    P8              6W /  119W |     786MiB /   8188MiB |     15%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import shutil
import torch
import random
import numpy as np
from tqdm import tqdm
from PIL import Image

# If you don't have a new enough diffusers, install/upgrade:
# pip install --upgrade diffusers transformers accelerate safetensors

from diffusers import StableDiffusionImg2ImgPipeline

# ------------------------------------------
# 0. Faster matmul on Ampere+ GPUs (optional)
# ------------------------------------------
torch.backends.cuda.matmul.allow_tf32 = True

# ------------------------------------------
# 1. Check if GPU supports float16
# ------------------------------------------
use_fp16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 7  # Ampere or newer

# ------------------------------------------
# 2. Load Stable Diffusion v1.5 in half-precision (Img2Img)
# ------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

if use_fp16:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        revision="fp16",   # ensures half-precision weights on Ampere+ (RTX 30 series, etc.)
        torch_dtype=torch.float16,
    ).to(device)
else:
    # Fallback to float32 if your GPU doesn't support half-precision
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float32,
    ).to(device)

# ------------------------------------------
# 3. Clear GPU cache (optional)
# ------------------------------------------
torch.cuda.empty_cache()

# ------------------------------------------
# 4. Category-specific prompts
# ------------------------------------------
category_prompts = {
    "cardboard": "A high-resolution image of a discarded, crushed cardboard box with torn edges, slightly stained and placed on a rough concrete surface.",
    "glass":     "A detailed photo of an empty, broken glass bottle lying on the ground with visible dirt and scratches.",
    "metal":     "A high-quality image of a used aluminum can with dents and scratches, slightly rusty, placed in an outdoor recycling bin.",
    "paper":     "A close-up of crumpled and torn waste paper, including newspapers and printed documents, slightly dirty and discarded.",
    "plastic":   "A realistic image of a plastic milk bottle, partially crushed, with dirt marks on it, lying on a sidewalk.",
    "trash":     "A pile of mixed waste including organic scraps, food leftovers, plastic wrappers, and used paper towels in a public trash bin."
}

# ------------------------------------------
# 5. Define dataset paths
# ------------------------------------------
input_dir = "../data/mlp_assignment/data/subset/train"
output_dir = "../data/dataset_diffusion_balanced02/train"

# Overwrite existing output directory
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------
# 6. Count images per category
# ------------------------------------------
class_counts = {}
for cat in os.listdir(input_dir):
    cat_path = os.path.join(input_dir, cat)
    if os.path.isdir(cat_path):
        class_counts[cat] = len(os.listdir(cat_path))

majority_class = max(class_counts, key=class_counts.get)
majority_size = class_counts[majority_class]
target_size = majority_size * 2  # Example: double the majority class

print(f"Class counts: {class_counts}")
print(f"Majority class: {majority_class} ({majority_size} images)")
print(f"Target size per class: {target_size}")

# ------------------------------------------
# 7. Augmentation parameters
# ------------------------------------------
strength = 0.5            # how much the new image differs from the original
guidance_scale = 7.5      # typical range for SD1.5 is ~7-8
image_size = (512, 512)   # desired output resolution
num_images_per_prompt = 1 # how many outputs per single input at a time

# ------------------------------------------
# 8. Preprocessing function
# ------------------------------------------
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize(image_size)
    return image

# ------------------------------------------
# 9. Function to check if an image is too dark / black
# ------------------------------------------
def is_black_image(image, mean_threshold=10):
    """
    Check if the image is predominantly black/dark by comparing
    the average pixel intensity to a threshold.
    """
    # Convert to grayscale
    gray = image.convert("L")
    arr = np.array(gray)

    # If the mean is below a small threshold, consider it 'black'
    return arr.mean() < mean_threshold

# ------------------------------------------
# 10. Balancing loop
# ------------------------------------------
for category, count in tqdm(class_counts.items(), desc="Balancing dataset"):
    class_path = os.path.join(input_dir, category)
    augmented_class_path = os.path.join(output_dir, category)
    os.makedirs(augmented_class_path, exist_ok=True)
    
    # 10a. Copy original images to new folder
    images = os.listdir(class_path)
    for img_name in images:
        src_path = os.path.join(class_path, img_name)
        dst_path = os.path.join(augmented_class_path, img_name)
        shutil.copy(src_path, dst_path)
    
    # 10b. Generate images until target size is reached
    num_needed = target_size - count
    
    # Use the category-specific prompt if available; else fall back to a trivial prompt
    prompt_text = category_prompts.get(category, ".")

    # We set an upper bound on the number of attempts to avoid infinite loops
    max_attempts = num_needed * 4  # e.g., 4 attempts for each needed image
    
    attempts = 0
    while num_needed > 0 and attempts < max_attempts:
        attempts += 1
        
        # Randomly pick an existing image in the category
        img_name = random.choice(images)
        img_path = os.path.join(class_path, img_name)
        
        # Preprocess input image (resize, convert to RGB)
        input_image = preprocess_image(img_path)
        
        # SDEdit-style generation with Img2Img
        with torch.no_grad():
            result = pipe(
                prompt=prompt_text,
                image=input_image,
                strength=strength,
                guidance_scale=guidance_scale,
                num_images_per_prompt=num_images_per_prompt,
            )

        synthetic_image = result.images[0]
        
        # Check if the image is black/dark
        if is_black_image(synthetic_image):
            # If it's black, skip saving and retry
            # You could also print a warning:
            # print(f"Warning: Black image generated for {category}, retrying...")
            continue
        
        # If the image is okay, save it
        base_name = os.path.splitext(img_name)[0]
        output_filename = f"{base_name}_aug_{num_needed}.png"
        output_path = os.path.join(augmented_class_path, output_filename)
        synthetic_image.save(output_path)
        
        # One synthetic image successfully generated
        num_needed -= 1

print(f"✅ Balanced dataset (SDEdit-style) saved at '{output_dir}'!")


In [23]:
import shutil

# Directory paths
input_dir = "../data/dataset_split/train"
output_dir = "../data/dataset_diffusion_balanced_02/train"

# Overwrite existing output directory (if you want a clean slate)
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Img2Img parameters
prompt = ""              # neutral or "photo of [class]"
strength = 0.3
guidance_scale = 4.0
image_size = (512, 512)  # resize input to this before generation
num_images_per_prompt = 1

# Utility: Preprocess input (resize, etc.)
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize(image_size)
    return image

# Utility: Filter out very dark/black images
def is_black_image(image, mean_threshold=10):
    gray = image.convert("L")
    arr = np.array(gray)
    return arr.mean() < mean_threshold


In [9]:
import os
import shutil
import random
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from diffusers import StableDiffusionImg2ImgPipeline, DDIMScheduler

# ----------------------------
# 1. Set Up Diffusion Pipeline
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.cuda.matmul.allow_tf32 = True  # Faster on Ampere+ GPUs

# Use half-precision (fp16) if supported
use_fp16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 7

if use_fp16:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        revision="fp16",
        torch_dtype=torch.float16,
    ).to(device)
else:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float32,
    ).to(device)

# Use DDIM scheduler for better control
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
torch.cuda.empty_cache()

# ----------------------------
# 2. Define Directories
# ----------------------------
input_dir = "../data/dataset_split/train"  # Path to your real waste dataset
output_dir = "../data/diffusion_augmented_dataset/train"  # Output directory for augmented images

# Clear output directory if it exists
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# ----------------------------
# 3. Utility Functions
# ----------------------------
image_size = (512, 512)  # High resolution for better quality

def preprocess_image(image_path):
    """Load and resize image to model's expected resolution."""
    image = Image.open(image_path).convert("RGB")
    return image.resize(image_size)

def is_invalid_image(image, mean_threshold=10):
    """Skip nearly blank or dark images."""
    gray = image.convert("L")
    arr = np.array(gray)
    return arr.mean() < mean_threshold

# ----------------------------
# 4. Improved Prompt Setup for Feature Enhancement
# ----------------------------
# Define prompts that add category-specific features (e.g., dirt, scratches, textures)
feature_prompts = {
    "cardboard": "A photo of a cardboard box with visible dirt and scratches, slightly crumpled.",
    "glass": "A photo of a glass bottle with visible cracks and dirt, lying on a rough surface.",
    "metal": "A photo of a metal can with dents and rust, placed in a recycling bin.",
    "paper": "A photo of crumpled paper with visible stains and tears, partially folded.",
    "plastic": "A photo of a plastic bottle with dirt marks and scratches, partially crushed.",
    "trash": "A photo of mixed waste with visible food scraps, plastic wrappers, and dirt."
}

def sample_prompt(category):
    """Generate a feature-enhancing prompt for the given category."""
    return feature_prompts.get(category, "A photo of waste.")

# ----------------------------
# 5. Diffusion Parameter Sampling
# ----------------------------
base_strength = 0.3  # Low strength to preserve the original image
base_guidance = 7.5  # Balanced guidance scale for better prompt adherence
fixed_inference_steps = 40  # More steps for better quality
fixed_eta = 0.6  # DDIM eta parameter

def sample_diffusion_params(category):
    strength = base_strength + random.uniform(-0.05, 0.05)  # ~0.25 to 0.35
    guidance_scale = base_guidance + random.uniform(-0.5, 0.5)  # ~7.0 to 8.0
    prompt = sample_prompt(category)
    return prompt, strength, guidance_scale, fixed_inference_steps, fixed_eta

# ----------------------------
# 6. Determine Target Count per Category
# ----------------------------
category_counts = {}
for category in os.listdir(input_dir):
    cat_path = os.path.join(input_dir, category)
    if os.path.isdir(cat_path):
        category_counts[category] = len(os.listdir(cat_path))
print("Real image counts per category:", category_counts)

max_count = max(category_counts.values())
target_count = max_count * 2  # Double the dataset size
print("Target images per category:", target_count)

# ----------------------------
# 7. Synthetic Data Generation Loop
# ----------------------------
for category, real_count in tqdm(category_counts.items(), desc="Augmenting categories"):
    cat_in = os.path.join(input_dir, category)
    cat_out = os.path.join(output_dir, category)
    os.makedirs(cat_out, exist_ok=True)
    
    # Copy original images into the augmented training set
    for file in os.listdir(cat_in):
        shutil.copy(os.path.join(cat_in, file), os.path.join(cat_out, file))
    
    synth_needed = target_count - real_count
    if synth_needed <= 0:
        continue

    print(f"Augmenting '{category}' with {synth_needed} synthetic images.")
    real_files = os.listdir(cat_in)
    attempts = 0
    max_attempts = synth_needed * 4

    while synth_needed > 0 and attempts < max_attempts:
        attempts += 1
        file_choice = random.choice(real_files)
        img_path = os.path.join(cat_in, file_choice)
        init_img = preprocess_image(img_path)
        
        prompt, strength, guidance, num_inference_steps, eta = sample_diffusion_params(category)
        
        with torch.no_grad():
            result = pipe(
                prompt=prompt,
                image=init_img,
                strength=strength,
                guidance_scale=guidance,
                num_inference_steps=num_inference_steps,
                eta=eta,
                num_images_per_prompt=1,
            )
        new_image = result.images[0]
        if is_invalid_image(new_image):
            continue
        
        base, _ = os.path.splitext(file_choice)
        out_filename = f"{base}_synth_{synth_needed}.png"
        out_filepath = os.path.join(cat_out, out_filename)
        print(f"Saving generated image for '{category}': {os.path.abspath(out_filepath)}")
        new_image.save(out_filepath)
        synth_needed -= 1

print("✅ Diffusion-based augmentation complete!")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Real image counts per category: {'cardboard': 368, 'glass': 336, 'metal': 632, 'paper': 400, 'plastic': 736, 'trash': 396}
Target images per category: 1472


Augmenting categories:   0%|          | 0/6 [00:00<?, ?it/s]

Augmenting 'cardboard' with 1104 synthetic images.


  0%|          | 0/11 [00:00<?, ?it/s]

Augmenting categories:   0%|          | 0/6 [00:23<?, ?it/s]


KeyboardInterrupt: 

In [1]:
import os
import shutil
import torch
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline

torch.backends.cuda.matmul.allow_tf32 = True

device = "cuda" if torch.cuda.is_available() else "cpu"

# Define category-specific configurations
category_settings = {
  "cardboard": {
    "prompt": "A perfectly clean, brand-new cardboard box, neatly presented in {shapes}. It has sharp edges with no creases, dirt, or damage. The object is isolated on a seamless, bright white studio backdrop with soft, even lighting.",
    "shapes": ["rectangular box", "square box", "flat folded box", "open-top box", "closed shipping box", "tall vertical box"],
    "strength": 0.8,
    "guidance_scale": 8,
    "image_size": [512, 512],
    "model": "runwayml/stable-diffusion-v1-5"
  },
  "glass": {
    "prompt": "A high-resolution image of a transparent, unbroken {items} with crisp reflections and pristine glass. The object is isolated on a seamless, bright white studio backdrop with soft, even lighting.",
    "items": ["glass bottle", "glass jar", "drinking glass", "wine glass", "glass tumbler", "glass vase"],
    "strength": 0.6,
    "guidance_scale": 11,
    "image_size": [512, 512],
    "model": "runwayml/stable-diffusion-v1-5"
  },
  "metal": {
    "prompt": "A detailed close-up shot of a {items}, entirely free of dents, rust, or any damage. The object showcases vibrant colors and sharp details, isolated against a seamless, bright white studio backdrop with soft, even lighting.",
    "items": ["aluminum can", "tin can", "food can", "soda can", "drink can", "beverage can"],
    "strength": 0.7,
    "guidance_scale": 15,
    "image_size": [512, 512],
    "model": "runwayml/stable-diffusion-v1-5"
  },
  "paper": {
    "prompt": "A single, clean printer {items} lying flat on a seamless, bright white studio backdrop. The edges are perfectly straight, and the surface is pristine, free of any folds, dirt, or creases.",
    "items": ["newspaper", "magazine", "printed document", "written paper sheet", "folded paper", "crumpled paper", "envelope"],
    "strength": 0.7,
    "guidance_scale": 14,
    "image_size": [512, 512],
    "model": "runwayml/stable-diffusion-v1-5"
  },
  "plastic": {
    "prompt": "A single plastic {items}, perfectly shaped with a smooth, glossy surface, free of any scratches or dents. The object is placed on an isolated, seamless, bright white studio backdrop with soft, even lighting.",
    "items": ["water bottle", "beverage bottle", "soda bottle", "juice bottle", "milk bottle"],
    "strength": 0.7,
    "guidance_scale": 12,
    "image_size": [512, 512],
    "model": "runwayml/stable-diffusion-v1-5"
  },
  "trash": {
    "prompt": "A random assortment of discarded waste items on a plain surface, including crumpled paper, torn plastic wrappers, empty snack pouches, and used napkins. The materials vary between glossy plastic, metallic foil, and soft paper, with some items showing creases, stains, or slight tears. The objects are casually scattered, resembling everyday trash left behind after a meal or snack. The lighting is even, highlighting textures and reflections on the different materials.",
    "strength": 0.8,
    "guidance_scale": 14,
    "image_size": [512, 512],
    "model": "runwayml/stable-diffusion-v1-5"
  }
}


# Define dataset paths
input_dir = "../data/subset/train"
output_dir = "../data/dataset_diffusion_balanced02/train"


if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Count images per category
class_counts = {cat: len(os.listdir(os.path.join(input_dir, cat))) for cat in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, cat))}

majority_class = max(class_counts, key=class_counts.get)
majority_size = class_counts[majority_class]
target_size = majority_size * 2  # Double the majority class

def preprocess_image(image_path, size):
    image = Image.open(image_path).convert("RGB")
    image = image.resize(size)
    return image

def is_black_image(image, mean_threshold=10):
    gray = image.convert("L")
    return np.array(gray).mean() < mean_threshold

for category, count in tqdm(class_counts.items(), desc="Balancing dataset"):
    settings = category_settings.get(category, category_settings["trash"])  # Default to "trash" settings if missing
    
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        settings["model"],
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    ).to(device)
    
    class_path = os.path.join(input_dir, category)
    augmented_class_path = os.path.join(output_dir, category)
    os.makedirs(augmented_class_path, exist_ok=True)
    
    images = os.listdir(class_path)
    for img_name in images:
        shutil.copy(os.path.join(class_path, img_name), os.path.join(augmented_class_path, img_name))
    
    num_needed = target_size - count
    print(num_needed)
    max_attempts = num_needed * 4
    attempts = 0
    
    while num_needed > 0 and attempts < max_attempts:
        attempts += 1
        img_name = random.choice(images)
        input_image = preprocess_image(os.path.join(class_path, img_name), settings["image_size"])
        
        with torch.no_grad():
            result = pipe(
                prompt=settings["prompt"],
                image=input_image,
                strength=settings["strength"],
                guidance_scale=settings["guidance_scale"],
                num_images_per_prompt=1,
            )
        
        synthetic_image = result.images[0]
        if is_black_image(synthetic_image):
            continue
        
        output_filename = f"{os.path.splitext(img_name)[0]}_aug_{num_needed}.png"
        synthetic_image.save(os.path.join(augmented_class_path, output_filename))
        num_needed -= 1

print(f"✅ Balanced dataset (Category-Specific Diffusion) saved at '{output_dir}'!")


Balancing dataset:   0%|          | 0/6 [00:00<?, ?it/s]Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

2


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['or snack . the lighting is even , highlighting textures and reflections on the different materials .']


  0%|          | 0/40 [00:00<?, ?it/s]

Balancing dataset: 100%|██████████| 6/6 [26:11<00:00, 261.84s/it]

✅ Balanced dataset (Category-Specific Diffusion) saved at '../data/dataset_diffusion_balanced02/train'!
